# Imports

In [1]:
!pip install python-dotenv

In [2]:
!cat /home/jovyan/.env

cat: /home/jovyan/.env: No such file or directory


In [9]:
import requests
from kafka import KafkaProducer
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time

In [10]:
# Charger les variables d'environnement depuis le fichier .env copié dans le conteneur
load_dotenv('/home/jovyan/.env')

# Configuration de l'API OpenSky
OPENSKY_URL = "https://opensky-network.org/api/states/all"
USERNAME = os.environ.get('OPENSKY_USERNAME')
PASSWORD = os.environ.get('OPENSKY_PASSWORD')

# Fonction pour envoyer les données OpenSky à Kafka

In [11]:

# Kafka configuration
kafka_config = {
    'bootstrap_servers': 'kafka1:9092',  # Update with your Kafka broker
}

# Initialize Kafka Producer
producer = KafkaProducer(
    bootstrap_servers="kafka1:9092",
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

def send_opensky_to_kafka(topic, url, username, password):

    # Fetch data from OpenSky
    response = requests.get(url, auth=(username, password))

    if response.status_code == 200:
        data = response.json()
        states = data.get("states", [])

        # Publish each state to Kafka
        for state in states:
            producer.send(topic, value=state)
            #print(f"Sent: {state}")

        # Ensure all messages are sent
        producer.flush()
        print(f"Sent {len(states)} records.")
    else:
        print(f"Failed to fetch data: {response.status_code}")

In [12]:
states = {
    "icao24": "icao24",
    "callsign": "callsign",
    "origin_country": "origin_country",
    "time_position": "time_position",
    "last_contact": "last_contact",
    "longitude": "longitude",
    "latitude": "latitude",
    "baro_altitude": "baro_altitude",
    "on_ground": "on_ground",
    "velocity": "velocity",
    "true_track": "true_track",
    "vertical_rate": "vertical_rate",
    "sensors": "sensors",
    "geo_altitude": "geo_altitude",
    "squawk": "squawk",
    "spi": "spi",
    "position_source": "position_source",
    "category": "category"
}

inital_date_str = "2025-01-25 16:00:00"

# Durée de récupération (minutes)
step = 60

# date initiale
initial_date_str = "2025-01-25 16:00:00"
# durée de récupération (minutes)
step = 60

date_format = "%Y-%m-%d %H:%M:%S"
initial_date = datetime.strptime(initial_date_str, date_format)
end_date = initial_date + timedelta(minutes=step)


start = initial_date.strftime(date_format)
end = end_date.strftime(date_format)




In [ ]:
# Send OpenSky data to Kafka

for i in range(10):
    print(f"{start}-{end}")
    send_opensky_to_kafka("opensky-flights", OPENSKY_URL, USERNAME, PASSWORD)
    time.sleep(5)
    initial_date = end_date
    end_date = initial_date + timedelta(minutes=step)
    start = initial_date.strftime(date_format)
    end = end_date.strftime(date_format)

In [14]:
TOPIC_ALL_FLIGHTS = "opensky_all_flights"
TOPIC_FILTERED_FLIGHTS = "opensky_filtered_flights"

current_time = 1735689600  # 2025-01-01 00:00:00 UTC

def send_filtered_flights_data(url, username, password ):
    """ Fetch flights for a specific timestamp and send them to Kafka. """
    global current_time  # Keep track of the simulated time
    params = {"time": current_time}  # Request data for this timestamp

    response = requests.get(OPENSKY_URL, params=params, auth=(USERNAME, PASSWORD))

    if response.status_code == 200:
        data = response.json()
        states = data.get("states", [])

        for state in states:
            # Convert array format to JSON format
            message = {
                "icao24": state[0],
                "callsign": state[1].strip(),
                "origin_country": state[2],
                "time_position": state[3],
                "last_contact": state[4],
                "longitude": state[5],
                "latitude": state[6],
                "altitude": state[7],
                "on_ground": state[8],
                "velocity": state[9],
                "heading": state[10],
                "vertical_rate": state[11],
                "sensors": state[12],
                "geo_altitude": state[13],
                "squawk": state[14],
                "spi": state[15],
                "position_source": state[16]
            }

            # Send data as JSON instead of an array
            producer.send(TOPIC_ALL_FLIGHTS, value=message)


            # Envoi des avions filtrés ( altitude > 10 000m)
            if state[7] and state[7] > 10000 and state[3] != None:  # Altitude (7ème index)
                producer.send(TOPIC_FILTERED_FLIGHTS, value=message)
                print(f"Sent to FILTERED_FLIGHTS: {message}")  # Debug

        producer.flush()
        print(f"Sent {len(states)} records.")
        current_time += 20
    else:
        print(f"Failed to fetch data: {response.status_code}")

In [16]:
import threading
import time

# Create a stop flag
stop_event = threading.Event()

def run_kafka_producer():
    """ Continuously send filtered flight data to Kafka every 20 seconds. """
    while not stop_event.is_set():
        send_filtered_flights_data(OPENSKY_URL, USERNAME, PASSWORD)
        time.sleep(1)

# Run the producer in a separate thread
producer_thread = threading.Thread(target=run_kafka_producer, daemon=True)
producer_thread.start()

print("Kafka producer is running in the background! Use `stop_kafka_producer()` to stop it.")

Kafka producer is running in the background! Use `stop_kafka_producer()` to stop it.


Failed to fetch data: 403
Failed to fetch data: 403
Failed to fetch data: 403
Failed to fetch data: 403
Failed to fetch data: 403
Failed to fetch data: 403
Failed to fetch data: 403
Failed to fetch data: 403
Failed to fetch data: 403


## Stop kafka thread

In [17]:
def stop_kafka_producer():
    """ Stops the Kafka producer thread gracefully. """
    print("Stopping Kafka producer...")
    stop_event.set()  # Set the stop flag
    producer_thread.join()  # Wait for the thread to finish
    print("Kafka producer stopped.")

# Call this function when you want to stop the producer
stop_kafka_producer()

Stopping Kafka producer...
Kafka producer stopped.


# Spark streaming 

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, window
from pyspark.sql.types import StructType, StringType, DoubleType

# Création de la session Spark
spark = SparkSession.builder \
    .appName("OpenSkySparkStreaming") \
.config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.3")    .getOrCreate()

print(spark.sparkContext.getConf().get("spark.jars.packages"))

# Define schema to match the JSON structure
schema = StructType() \
    .add("icao24", StringType()) \
    .add("callsign", StringType()) \
    .add("origin_country", StringType()) \
    .add("time_position", DoubleType()) \
    .add("last_contact", DoubleType()) \
    .add("longitude", DoubleType()) \
    .add("latitude", DoubleType()) \
    .add("altitude", DoubleType()) \
    .add("on_ground", BooleanType()) \
    .add("velocity", DoubleType()) \
    .add("heading", DoubleType()) \
    .add("vertical_rate", DoubleType()) \
    .add("sensors", StringType()) \
    .add("geo_altitude", DoubleType()) \
    .add("squawk", StringType()) \
    .add("spi", BooleanType()) \
    .add("position_source", StringType())

# Create Spark session
spark = SparkSession.builder \
    .appName("OpenSkyFlightsStreaming") \
    .getOrCreate()

# Read from Kafka (ALL flights)
df_all_flights = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka1:9092") \
    .option("subscribe", "opensky_all_flights") \
    .option("startingOffsets", "earliest") \
    .load()

# Read from Kafka (FILTERED flights)
df_filtered_flights = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka1:9092") \
    .option("subscribe", "opensky_filtered_flights") \
    .option("startingOffsets", "earliest") \
    .load()

# Convert Kafka messages from bytes to JSON (ALL flights)
df_all_flights = df_all_flights.selectExpr("CAST(value AS STRING) as json_value") \
    .select(from_json(col("json_value"), schema).alias("data")) \
    .select("data.*") \
    .withColumn("event_time", current_timestamp())

# Convert Kafka messages from bytes to JSON (FILTERED flights)
df_filtered_flights = df_filtered_flights.selectExpr("CAST(value AS STRING) as json_value") \
    .select(from_json(col("json_value"), schema).alias("data")) \
    .select("data.*") \
    .withColumn("event_time", current_timestamp())

# Apply windowing (1-minute aggregation) for ALL flights
df_all_windowed = df_all_flights.groupBy(
    window(col("event_time"), "1 minute"),
    col("origin_country")
).count()

# Apply windowing (1-minute aggregation) for FILTERED flights
df_filtered_windowed = df_filtered_flights.groupBy(
    window(col("event_time"), "1 minute"),
    col("origin_country")
).count()

# Display output in console (ALL flights)
query1 = df_all_windowed.writeStream \
    .outputMode("update") \
    .format("console") \
    .start()

# Display output in console (FILTERED flights)
query2 = df_filtered_windowed.writeStream \
    .outputMode("update") \
    .format("console") \
    .start()

query1.awaitTermination()
query2.awaitTermination()